In [2]:
import json
import openai
from pprint import pprint
from dotenv import load_dotenv
import os
import pandas as pd
import random
import requests
import ast
from collections import defaultdict

TRAIN_DATA_FILE = "train.json"
PROMPT_FILE = "prompt.txt"

In [3]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [4]:
def read_data(data_file): 
    with open(data_file, "r") as f: 
        return json.load(f)

def process_data_to_row(data_point): 
    return {
        "Conversation Context": '\n'.join(data_point[0].split('\n')[:-1]),
        "Helper Response": data_point[0].split('\n')[-1], 
        "Better Response": data_point[1]['alternative'],
        "Feedback": data_point[1]['feedback'], 
        "Good Areas": data_point[1]['goodareas'],
        "Bad Areas": data_point[1]['badareas']
    }

In [5]:
def parse_data_point(data_point):
    data_text = data_point['text']
    keys = data_text.split('###')

    # Extract the instruction, input, and response
    instruction = keys[1].strip()
    input_text = keys[2].strip()
    response_text = keys[3].strip()

    # Extract the conversation and helper indices
    conv_index = data_point['conv_index']
    helper_index = data_point['helper_index']

    # Format the input to remove the first line
    input_lines = input_text.split('\n')
    input_lines = input_lines[1:]
    input_formatted = [line.strip() for line in input_lines if line.strip()]  # Remove empty lines and extra spaces
    

    # Parse the response text
    response_data = json.loads(response_text[response_text.index(":") + 1:].strip())

    return {
        'conv_index': conv_index,
        'helper_index': helper_index,
        'instruction': instruction,
        'input': input_formatted,
        'response': response_data
    }

In [8]:
data = read_data(TRAIN_DATA_FILE)
c_result = []
sorted_result = []
for c_index in range(1, 3):
    for data_point in data:
        data_text = data_point['text']
        keys = data_text.split('###')
        if data_point['conv_index'] == c_index:
            c_result.append(parse_data_point(data_point))
    c_result = sorted(c_result, key = lambda x: x['helper_index'])
    sorted_result += c_result
processed_dict = [result for result in sorted_result if not result['response']['perfect'] and ('Questions' in result['response']['badareas'] or 'Suggestions' in result['response']['badareas'])]
print(processed_dict)

[{'conv_index': 1, 'helper_index': 2, 'instruction': "Instruction:\nGive feedback to the Helper's last response.", 'input': ['Seeker: Hey', 'Helper: HI, can I help today?', 'Seeker: I have been married for 5 years but i am very unhappy and anxious. I want to break up with my partner but not sure how, this is increasing my anxiety', 'Helper: I am sorry to hear that. I know it can be really stressful when you are considering a break up. How long have you thought about doing this?', "Seeker: i've thought about this for the last year, things have been going downhill since. I think its starting to become noticible", 'Helper: That is a long time to think about something, so you have given it a lot of time and thought. Have you mentioned it to your partner?'], 'response': {'perfect': False, 'goodareas': ['Empathy'], 'feedback': "The goal is to empathize and validate the seeker's feelings and provide a supportive, judgment-free environment. It might be better to first validate the seeker's fee

In [9]:
# with open(f'{c_index}_processed_data.json', 'w') as json_file:
#     json.dump(sorted_result, json_file, indent=4)
# print(f"Data has been saved to {c_index}_processed_data.json")

# with open(f'{c_index}_flawed_response.json', 'w') as json_file:
#     json.dump(flawed_response, json_file, indent=4)
# print(f"Data has been saved to {c_index}_flawed_response.json")

with open('processed_dict.json', 'w') as json_file:
    json.dump(processed_dict, json_file, indent=4)
print(f"Data has been saved to processed_dict.json")

Data has been saved to processed_dict.json


In [15]:
def find_perf_data_points(data, imperfect, c_index): 
    for points in imperfect:
        if points[0]['conv_index'] == c_index:
            return process_data_to_row(points[0])
        

In [16]:
find_perf_data_points(data, imperfect, 127)

NameError: name 'imperfect' is not defined

In [ ]:
# def group_and_sort_data_points(data, c_index=None):
#     grouped_data = defaultdict(list)
    
#     # Parse each data point and group by conv_index
#     for data_point in data:
#         parsed_data = parse_data_point(data_point, c_index)
#         if parsed_data:
            
            
    
#     # Sort each group by helper_index
#     for conv_index in grouped_data:
#         grouped_data[conv_index].sort(key=lambda x: x['helper_index'])
    
#     return grouped_data